In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import autokeras as ak
import os
import tensorflow as tf

In [2]:
data = np.load('/work/n1000000_0804_all_flat.npz')

In [3]:
input_state = 2
target = data['theta23']
if input_state == 0:
    data_all = data['ve_dune']
elif input_state == 1:
    data_all = np.column_stack([data['ve_dune'], data['vu_dune'], data['vebar_dune'], data['vubar_dune']])
elif input_state == 2:
    data_all = np.column_stack([data['ve_dune'], data['vu_dune'], data['vebar_dune'], data['vubar_dune'], data['ve_t2hk'], data['vu_t2hk'], data['vebar_t2hk'], data['vubar_t2hk']])

x_train = data_all[:10000]
y_train = target[:10000]
x_train2 = data_all[10000:900000]
y_train2 = target[10000:900000]
x_test = data_all[900000:]
y_test = target[900000:]

In [4]:
clf = ak.StructuredDataRegressor(overwrite=True, max_trials=50)
clf.fit(x_train, y_train,
           validation_split = 0.1,
           batch_size=64,
           epochs=20,
           verbose=1,
           shuffle = True
       )

Trial 50 Complete [00h 00m 34s]
val_loss: 1.4491512775421143

Best val_loss So Far: 0.016008101403713226
Total elapsed time: 00h 28m 32s
INFO:tensorflow:Oracle triggered exit

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1/20
157/157 [==============================] - 1s 8ms/step - loss: 138.6190 - mean_squared_error: 138.6190
Epoch 2/20
157/157 [==============================] - 1s 8ms/step - loss: 1.3585 - mean_squared_error: 1.3585
Epoch 3/20
157/157 [==============================] - 1s 8ms/step - loss: 0.3738 - mean_squared_error: 0.3738

In [5]:
model = clf.export_model()
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 244)]             0         
_________________________________________________________________
multi_category_encoding (Mul (None, 244)               0         
_________________________________________________________________
normalization (Normalization (None, 244)               489       
_________________________________________________________________
dense (Dense)                (None, 1024)              250880    
_________________________________________________________________
re_lu (ReLU)                 (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                32800     
_________________________________________________________________
re_lu_1 (ReLU)               (None, 32)               

In [6]:
model.fit(x_train2, y_train2,
           validation_split = 0.1,
           batch_size=64,
           epochs=20,
           verbose=1,
           shuffle = True
         )

Epoch 1/20
12516/12516 [==============================] - 103s 8ms/step - loss: 0.1089 - mean_squared_error: 0.1089 - val_loss: 0.0344 - val_mean_squared_error: 0.0344
Epoch 2/20
12516/12516 [==============================] - 101s 8ms/step - loss: 0.0449 - mean_squared_error: 0.0449 - val_loss: 0.0218 - val_mean_squared_error: 0.0218
Epoch 3/20
12516/12516 [==============================] - 101s 8ms/step - loss: 0.0253 - mean_squared_error: 0.0253 - val_loss: 0.0061 - val_mean_squared_error: 0.0061
Epoch 4/20
12516/12516 [==============================] - 101s 8ms/step - loss: 0.0203 - mean_squared_error: 0.0203 - val_loss: 0.0028 - val_mean_squared_error: 0.0028
Epoch 5/20
12516/12516 [==============================] - 100s 8ms/step - loss: 0.0163 - mean_squared_error: 0.0163 - val_loss: 0.0014 - val_mean_squared_error: 0.0014
Epoch 6/20
12516/12516 [==============================] - 100s 8ms/step - loss: 0.0128 - mean_squared_error: 0.0128 - val_loss: 0.0020 - val_mean_squared_error:

In [ ]:
model.evaluate(x_test, y_test)
pre_test = model.predict(x_test)

In [ ]:
plt.hist(pre_test, histtype = 'step', bins = 100, label = 'prediction')
plt.hist(y_test, histtype = 'step', bins = 100, label = 'target')
plt.xlabel('theta23')
plt.ylabel('number')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(7,7))
plt.plot(pre_test, y_test, '.', alpha = 0.05)
plt.xlabel('prediction')
plt.ylabel('target')
plt.title('theta23')

In [ ]:
if input_state == 0:
    modelname = '/work/Regression/models_ve_dune/0803_theta23_1.h5'
    if not os.path.isfile(modelname):
        model.save(modelname)
elif input_state == 1:
    modelname = '/work/Regression/models_all_dune/0803_theta23_1.h5'
    if not os.path.isfile(modelname):
        model.save(modelname)
elif input_state == 2:
    modelname = '/work/Regression/models_all/0804_theta23_1.h5'
    if not os.path.isfile(modelname):
        model.save(modelname)